# 완료를 스트리밍하는 방법

기본적으로 OpenAI에 완료를 요청하면 전체 완료가 생성된 후 단일 응답으로 다시 전송됩니다.

긴 완료를 생성하는 경우 응답을 기다리는 데 몇 초가 걸릴 수 있습니다.

응답을 더 빨리 받으려면 완료가 생성되는 동안 완료를 '스트리밍'할 수 있습니다. 이렇게 하면 전체 완료가 완료되기 전에 완료의 시작 부분부터 인쇄 또는 처리를 시작할 수 있습니다.

완료를 스트리밍하려면 채팅 완료 또는 완료 엔드포인트를 호출할 때 `stream=True`를 설정하세요. 그러면 응답을 [데이터 전용 서버 전송 이벤트](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format)로 스트리밍하는 객체가 반환됩니다. `메시지` 필드가 아닌 `델타` 필드에서 청크를 추출합니다.


---------


By default, when you request a completion from the OpenAI, the entire completion is generated before being sent back in a single response.

If you're generating long completions, waiting for the response can take many seconds.

To get responses sooner, you can 'stream' the completion as it's being generated. This allows you to start printing or processing the beginning of the completion before the full completion is finished.

To stream completions, set `stream=True` when calling the chat completions or completions endpoints. This will return an object that streams back the response as [data-only server-sent events](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format). Extract chunks from the `delta` field rather than the `message` field.

## 단점

프로덕션 애플리케이션에서 `stream=True`를 사용하면 부분 완료를 평가하기가 더 어려울 수 있으므로 완료 내용을 조정하기가 더 어려워집니다. 이는 [승인된 사용](https://beta.openai.com/docs/usage-guidelines)에 영향을 미칩니다.

스트리밍 응답의 또 다른 작은 단점은 응답에 더 이상 얼마나 많은 토큰이 소비되었는지 알려주는 '사용량' 필드가 포함되지 않는다는 것입니다. 모든 응답을 수신하고 결합한 후, [`tiktoken`](How_to_count_tokens_with_tiktoken.ipynb)를 사용하여 직접 계산할 수 있습니다.

-----------

Note that using `stream=True` in a production application makes it more difficult to moderate the content of the completions, as partial completions may be more difficult to evaluate. which has implications for [approved usage](https://beta.openai.com/docs/usage-guidelines).

Another small drawback of streaming responses is that the response no longer includes the `usage` field to tell you how many tokens were consumed. After receiving and combining all of the responses, you can calculate this yourself using [`tiktoken`](How_to_count_tokens_with_tiktoken.ipynb).

## 예시 코드

아래는 이 노트북의 예시입니다:
1. 일반적인 채팅 완료 응답의 모습
2. 스트리밍 채팅 완료 응답의 모습 2.
3. 채팅 완료를 스트리밍하면 얼마나 많은 시간이 절약되나요?
4. 비채팅 완료를 스트리밍하는 방법(`text-davinci-003`과 같은 이전 모델에서 사용)

-----------

Below, this notebook shows:
1. What a typical chat completion response looks like
2. What a streaming chat completion response looks like
3. How much time is saved by streaming a chat completion
4. How to stream non-chat completions (used by older models like `text-davinci-003`)

In [1]:
# imports
import openai  # for OpenAI API calls
import time  # for measuring time duration of API calls

### 1. 일반적인 채팅 완료 응답의 모습

일반적인 ChatCompletions API 호출에서는 먼저 응답을 계산한 다음 한꺼번에 반환합니다.

----------

With a typical ChatCompletions API call, the response is first computed and then returned all at once.

In [3]:
# Example of an OpenAI ChatCompletion request
# https://platform.openai.com/docs/guides/chat

# record the time before the request is sent
start_time = time.time()

# send a ChatCompletion request to count to 100
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'user', 'content': 'Count to 100, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ],
    temperature=0,
)

# calculate the time it took to receive the response
response_time = time.time() - start_time

# print the time delay and text received
print(f"Full response received {response_time:.2f} seconds after request")
print(f"Full response received:\n{response}")


Full response received 19.67 seconds after request
Full response received:
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\n1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100.",
        "role": "assistant"
      }
    }
  ],
  "created": 1678746299,
  "id": "chatcmpl-6tkt9P4UOp7agE7q2QmQjLnJtgocg",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 301,
    "prompt_tokens": 36,
    "total_tokens": 337
  }
}


응답은 `response['choices'][0]['message']`로 추출할 수 있습니다.

응답의 내용은 `response['choices'][0]['message']['content']`로 추출할 수 있습니다.

-------------

The reply can be extracted with `response['choices'][0]['message']`.

The content of the reply can be extracted with `response['choices'][0]['message']['content']`.

In [5]:
reply = response['choices'][0]['message']
print(f"Extracted reply: \n{reply}")

reply_content = response['choices'][0]['message']['content']
print(f"Extracted content: \n{reply_content}")


Extracted reply: 
{
  "content": "\n\n1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100.",
  "role": "assistant"
}
Extracted content: 


1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100.


### 2. 채팅 완료를 스트리밍하는 방법

스트리밍 API 호출을 사용하면 [이벤트 스트림](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format)을 통해 응답이 청크 단위로 점진적으로 전송됩니다. Python에서는 `for` 루프를 사용하여 이러한 이벤트를 반복할 수 있습니다.

어떤 모습인지 살펴봅시다:

--------------------

With a streaming API call, the response is sent back incrementally in chunks via an [event stream](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format). In Python, you can iterate over these events with a `for` loop.

Let's see what it looks like:

In [6]:
# Example of an OpenAI ChatCompletion request with stream=True
# https://platform.openai.com/docs/guides/chat

# a ChatCompletion request
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'user', 'content': "What's 1+1? Answer in one word."}
    ],
    temperature=0,
    stream=True  # this time, we set stream=True
)

for chunk in response:
    print(chunk)

{
  "choices": [
    {
      "delta": {
        "role": "assistant"
      },
      "finish_reason": null,
      "index": 0
    }
  ],
  "created": 1678746571,
  "id": "chatcmpl-6tkxXDCILeWMMAD0kX382uC9DjUMN",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion.chunk"
}
{
  "choices": [
    {
      "delta": {
        "content": "\n\n"
      },
      "finish_reason": null,
      "index": 0
    }
  ],
  "created": 1678746571,
  "id": "chatcmpl-6tkxXDCILeWMMAD0kX382uC9DjUMN",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion.chunk"
}
{
  "choices": [
    {
      "delta": {
        "content": "2"
      },
      "finish_reason": null,
      "index": 0
    }
  ],
  "created": 1678746571,
  "id": "chatcmpl-6tkxXDCILeWMMAD0kX382uC9DjUMN",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion.chunk"
}
{
  "choices": [
    {
      "delta": {},
      "finish_reason": "stop",
      "index": 0
    }
  ],
  "created": 1678746571,
  "id": "chatcmpl-6tkxXDCILeWMMAD0kX382uC

위에서 볼 수 있듯이 스트리밍 응답에는 `메시지` 필드가 아닌 `델타` 필드가 있습니다. 델타`는 다음과 같은 것들을 담을 수 있습니다:
- 역할 토큰(예: `{"role": "assistant"}`)
- 콘텐츠 토큰(예: `{"content": "\n\n"}`)
- 스트림이 끝난 경우, 아무것도 없음(예: `{}`)

-------------

As you can see above, streaming responses have a `delta` field rather than a `message` field. `delta` can hold things like:
- a role token (e.g., `{"role": "assistant"}`)
- a content token (e.g., `{"content": "\n\n"}`)
- nothing (e.g., `{}`), when the stream is over

### 3. 채팅 완료 스트리밍으로 절약되는 시간

이제 `gpt-3.5-turbo`에게 다시 100까지 세도록 요청하고 시간이 얼마나 걸리는지 알아봅시다.

-----------------

Now let's ask `gpt-3.5-turbo` to count to 100 again, and see how long it takes.

In [7]:
# Example of an OpenAI ChatCompletion request with stream=True
# https://platform.openai.com/docs/guides/chat

# record the time before the request is sent
start_time = time.time()

# send a ChatCompletion request to count to 100
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'user', 'content': 'Count to 100, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ],
    temperature=0,
    stream=True  # again, we set stream=True
)

# create variables to collect the stream of chunks
collected_chunks = []
collected_messages = []
# iterate through the stream of events
for chunk in response:
    chunk_time = time.time() - start_time  # calculate the time delay of the chunk
    collected_chunks.append(chunk)  # save the event response
    chunk_message = chunk['choices'][0]['delta']  # extract the message
    collected_messages.append(chunk_message)  # save the message
    print(f"Message received {chunk_time:.2f} seconds after request: {chunk_message}")  # print the delay and text

# print the time delay and text received
print(f"Full response received {chunk_time:.2f} seconds after request")
full_reply_content = ''.join([m.get('content', '') for m in collected_messages])
print(f"Full conversation received: {full_reply_content}")


Message received 1.57 seconds after request: {
  "role": "assistant"
}
Message received 1.57 seconds after request: {
  "content": "\n\n"
}
Message received 1.57 seconds after request: {
  "content": "1"
}
Message received 1.63 seconds after request: {
  "content": ","
}
Message received 1.70 seconds after request: {
  "content": " "
}
Message received 1.80 seconds after request: {
  "content": "2"
}
Message received 1.93 seconds after request: {
  "content": ","
}
Message received 1.93 seconds after request: {
  "content": " "
}
Message received 2.00 seconds after request: {
  "content": "3"
}
Message received 2.06 seconds after request: {
  "content": ","
}
Message received 2.12 seconds after request: {
  "content": " "
}
Message received 2.55 seconds after request: {
  "content": "4"
}
Message received 2.55 seconds after request: {
  "content": ","
}
Message received 2.55 seconds after request: {
  "content": " "
}
Message received 2.55 seconds after request: {
  "content": "5"
}
Me

Message received 9.80 seconds after request: {
  "content": "42"
}
Message received 9.80 seconds after request: {
  "content": ","
}
Message received 9.80 seconds after request: {
  "content": " "
}
Message received 9.83 seconds after request: {
  "content": "43"
}
Message received 9.89 seconds after request: {
  "content": ","
}
Message received 9.95 seconds after request: {
  "content": " "
}
Message received 10.03 seconds after request: {
  "content": "44"
}
Message received 10.10 seconds after request: {
  "content": ","
}
Message received 10.19 seconds after request: {
  "content": " "
}
Message received 10.20 seconds after request: {
  "content": "45"
}
Message received 10.32 seconds after request: {
  "content": ","
}
Message received 10.32 seconds after request: {
  "content": " "
}
Message received 10.38 seconds after request: {
  "content": "46"
}
Message received 10.44 seconds after request: {
  "content": ","
}
Message received 10.50 seconds after request: {
  "content": " 

Message received 17.40 seconds after request: {
  "content": " "
}
Message received 17.67 seconds after request: {
  "content": "83"
}
Message received 17.67 seconds after request: {
  "content": ","
}
Message received 17.67 seconds after request: {
  "content": " "
}
Message received 17.67 seconds after request: {
  "content": "84"
}
Message received 17.71 seconds after request: {
  "content": ","
}
Message received 17.77 seconds after request: {
  "content": " "
}
Message received 17.82 seconds after request: {
  "content": "85"
}
Message received 17.89 seconds after request: {
  "content": ","
}
Message received 17.94 seconds after request: {
  "content": " "
}
Message received 18.19 seconds after request: {
  "content": "86"
}
Message received 18.19 seconds after request: {
  "content": ","
}
Message received 18.19 seconds after request: {
  "content": " "
}
Message received 18.19 seconds after request: {
  "content": "87"
}
Message received 18.24 seconds after request: {
  "conten

#### 시간 비교

위의 예에서 두 요청 모두 완전히 완료되는 데 약 3초가 걸렸습니다. 요청 시간은 로드 및 기타 확률적 요인에 따라 달라질 수 있습니다.

하지만 스트리밍 요청의 경우 0.1초 후에 첫 번째 토큰을 받았고, 이후 토큰은 약 0.01~0.02초마다 받았습니다.

--------------

In the example above, both requests took about 3 seconds to fully complete. Request times will vary depending on load and other stochastic factors.

However, with the streaming request, we received the first token after 0.1 seconds, and subsequent tokens every ~0.01-0.02 seconds.

### 4. 채팅이 아닌 완료 내용을 스트리밍하는 방법(`text-davinci-003`과 같은 구형 모델에서 사용)

#### 일반적인 완성 요청

일반적인 완성 API 호출에서는 먼저 텍스트를 계산한 다음 한꺼번에 반환합니다.

----------

With a typical Completions API call, the text is first computed and then returned all at once.

In [8]:
# Example of an OpenAI Completion request
# https://beta.openai.com/docs/api-reference/completions/create

# record the time before the request is sent
start_time = time.time()

# send a Completion request to count to 100
response = openai.Completion.create(
    model='text-davinci-002',
    prompt='1,2,3,',
    max_tokens=193,
    temperature=0,
)

# calculate the time it took to receive the response
response_time = time.time() - start_time

# extract the text from the response
completion_text = response['choices'][0]['text']

# print the time delay and text received
print(f"Full response received {response_time:.2f} seconds after request")
print(f"Full text received: {completion_text}")

Full response received 3.95 seconds after request
Full text received: 4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100


#### 스트리밍 완료 요청

스트리밍 완료 API를 호출하면 일련의 이벤트를 통해 텍스트가 다시 전송됩니다. Python에서는 `for` 루프를 사용하여 이러한 이벤트를 반복할 수 있습니다.

----------

With a streaming Completions API call, the text is sent back via a series of events. In Python, you can iterate over these events with a `for` loop.

In [9]:
# Example of an OpenAI Completion request, using the stream=True option
# https://beta.openai.com/docs/api-reference/completions/create

# record the time before the request is sent
start_time = time.time()

# send a Completion request to count to 100
response = openai.Completion.create(
    model='text-davinci-002',
    prompt='1,2,3,',
    max_tokens=193,
    temperature=0,
    stream=True,  # this time, we set stream=True
)

# create variables to collect the stream of events
collected_events = []
completion_text = ''
# iterate through the stream of events
for event in response:
    event_time = time.time() - start_time  # calculate the time delay of the event
    collected_events.append(event)  # save the event response
    event_text = event['choices'][0]['text']  # extract the text
    completion_text += event_text  # append the text
    print(f"Text received: {event_text} ({event_time:.2f} seconds after request)")  # print the delay and text

# print the time delay and text received
print(f"Full response received {event_time:.2f} seconds after request")
print(f"Full text received: {completion_text}")

Text received: 4 (0.53 seconds after request)
Text received: , (0.53 seconds after request)
Text received: 5 (0.53 seconds after request)
Text received: , (0.53 seconds after request)
Text received: 6 (0.53 seconds after request)
Text received: , (0.53 seconds after request)
Text received: 7 (0.53 seconds after request)
Text received: , (0.53 seconds after request)
Text received: 8 (0.53 seconds after request)
Text received: , (0.53 seconds after request)
Text received: 9 (0.70 seconds after request)
Text received: , (0.70 seconds after request)
Text received: 10 (0.70 seconds after request)
Text received: , (0.70 seconds after request)
Text received: 11 (0.70 seconds after request)
Text received: , (0.70 seconds after request)
Text received: 12 (0.70 seconds after request)
Text received: , (0.70 seconds after request)
Text received: 13 (0.71 seconds after request)
Text received: , (0.71 seconds after request)
Text received: 14 (0.71 seconds after request)
Text received: , (0.71 second

#### 시간 비교

위의 예에서 두 요청 모두 완전히 완료되는 데 약 3초가 걸렸습니다. 요청 시간은 로드 및 기타 확률적 요인에 따라 달라질 수 있습니다.

그러나 스트리밍 요청의 경우 0.18초 후에 첫 번째 토큰을 받았고, 이후 토큰은 약 0.01~0.02초마다 받았습니다.

-----------

In the example above, both requests took about 3 seconds to fully complete. Request times will vary depending on load and other stochastic factors.

However, with the streaming request, we received the first token after 0.18 seconds, and subsequent tokens every ~0.01-0.02 seconds.